# Writeup | Vehicle tracking
---
[![Udacity](https://s3.amazonaws.com/udacity-sdc/github/shield-carnd.svg)](http://www.udacity.com/drive)

_**Note for the reviewer:** the video frame stops/freezes three times for 0.5 second. It is because I had to cut the video in three pieces. My laptop is not powerful enough to process the all video at once.  Please accept my apologies for this inconvenient. `May I ask you to mail me the feedback review by email (chatmoon@gmail.com) please? Thank you, #mo` _

**Abstract — This notebook is the writeup of the Vehicle Tracking project**  as part of the SELF-DRIVING CAR nanodegree program. We apply a combinaison of machine learning and computer vision techniques using `OpenCV` and `scikit-learn` functions to the task of vehicle tracking in a video from a front-facing camera on a car. To that end, we use
Support Vector Machine (SVM) Classifier, Histogram of Gradients (HOG), Spatial binary and Color Histogram techniques.   

The project is broken down into the following steps:
* Extract histogram of oriented gradients (HOG) feature vector
* Train support vector machine (SVM) classifier
* Normalize the features and randomize a selection for training and testing
* Detect vehicles frame by frame
* Reject outliers creating a heat map of recurring detections frame by frame 
* Track vehicle and estimate a bounding box for vehicles detected 


Here I will consider [**the rubric points**](https://review.udacity.com/#!/rubrics/513/view) individually and describe how I addressed each point in my implementation.

---
### Histogram of Oriented Gradients (HOG)
---
#### 1. Extract histogram of oriented gradients (HOG) feature vector from the training images

*The code for this part is contained in lines 30 through 151 of the file called `main.py`. It is split into several functions, i.e. `get_hog_features()`, `bin_spatial()`, `color_hist()`, in lines 60 through 151. There are all combined in the function called `extract_features()`.*  

I used a dataset that has two types of image and label: **8,793** `RGB` images with car and **8,968** images without car. I started by reading in all the vehicle and non-vehicle images. The code listing all the images is contained in lines 30 through 56 of the file `main.py`, in the `list_files()` and `list_all_images()` methods.

Here is an example of five of each of the `vehicle` and `non-vehicle` classes:
![fig.0](https://raw.githubusercontent.com/chatmoon/SDC_PRJ05_Vehicle_Detection/master/output_images/_sample%20of%20cars%20and%20notcars_5%20items.png)

The function `get_hog_features()` extracts the HOG feature vector:   

```python
# -- lines 60 through 77 of the file main.py --
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):    
    if vis==True: # call with two outputs if vis==True
        features, hog_image = hog(img,orientations=orient,
                                  pixels_per_cell = (pix_per_cell, pix_per_cell),
                                  cells_per_block = (cell_per_block, cell_per_block),
                                  transform_sqrt  = True,
                                  visualize = vis, feature_vector = feature_vec )
        return features, hog_image
    
    else: # otherwise call with one output
        features = hog(img, orientations=orient,
                                  pixels_per_cell = (pix_per_cell, pix_per_cell),
                                  cells_per_block = (cell_per_block, cell_per_block),
                                  transform_sqrt  = True,
                                  visualize = vis, feature_vector = feature_vec )
        return features
```

I then explored different color spaces and different `skimage.hog()` parameters (`orientations`, `pixels_per_cell`, and `cells_per_block`).

I grabbed random images from each of the two classes and displayed them to get a feel for what the `skimage.hog()` output looks like. Here is an example using the `RGB` color space and HOG parameters of `orientations=6`, `pixels_per_cell=(8, 8)` and `cells_per_block=(2, 2)`:
![fig.1](https://raw.githubusercontent.com/chatmoon/SDC_PRJ05_Vehicle_Detection/master/output_images/_hog_image_orient-6.png)

The function `bin_spatial()` extracts the binned color features:   
```python
# -- lines 80 through 86 of the file main.py --
def bin_spatial(img, size=(32, 32)):
    # create the feature vector
    color1 = cv2.resize(img[:, :, 0], size).ravel()
    color2 = cv2.resize(img[:, :, 1], size).ravel()
    color3 = cv2.resize(img[:, :, 2], size).ravel()
    return np.hstack((color1, color2, color3))
```

The hog features and the binned color features are combined. They complement each other in the information they capture about the vehicle. Subsequently, I will also combine the histograms of color features.

The function `color_hist()` extracts the histogram color features:
```python
# -- lines 90 through 98 of the file main.py --
def color_hist(img, nbins=32):
    # compute the histogram of the color channels separately
    channel1_hist = np.histogram( img[:, :, 0], bins=nbins )
    channel2_hist = np.histogram( img[:, :, 1], bins=nbins )
    channel3_hist = np.histogram( img[:, :, 2], bins=nbins )
    # concatenate the histograms into a single feautre vector
    hist_features  = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # return the individual histograms, bin_centers and feature vector
    return hist_features
```

These feautres are all combined into a main feature vector by the `extract_features()` method:
```python
# -- lines 102 through 151 of the file main.py --
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32), hist_bins=32, orient=9, pix_per_cell=8, 
                     cell_per_block=2, hog_channel=0, spatial_feat=True, hist_feat=True, hog_feat=True):    
    features = []  # create a list to append feature vectors to
    for file in imgs: # iterate through the list of images
        file_features = []        
        image = mpimg.imread(file) # read in each one by one
        feature_image = convert_color(image, conv=color_space) # apply color conversation if other than 'RGB'
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:            
            hist_features = color_hist(feature_image, nbins=hist_bins) # apply color_hist()
            file_features.append(hist_features)
        if hog_feat == True:
            if hog_channel == 'ALL': # call get_hog_feature() with vis=False, feature_vec=True
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], orient,
                                        pix_per_cell, cell_per_block, vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient,
                                                pix_per_cell, cell_per_block, vis=False, feature_vec=True)           
            file_features.append(hog_features) # append the new feature vector to the features list
        features.append(np.concatenate(file_features))
    # return list of feature vectors
    return features
```

All combined features make the vehicle dection algorithm more robust.

#### 2. Explain how you settled on your final choice of HOG parameters.

I tried various combinations of parameters to optimize at best both the accuracy and the sum of the feature computation time and the SVC training time. For this purpose, I created the `exp_hog_parameters()` function, lines 19 through 95 of the `hog_parameters.py` file. I applied a color transform and append binned color features, as well as histograms of color, to the HOG feature vector.  

_**Note:** I did these experiments with the whole dataset instead of a smaller one. It was a mistake. I would be able to shorter the time spent in this part of the project._

Here are a few results of these various combinations of parameters:

> a. _various color space_:   

| color_space | cell_per_block| hist_bins | hist_feat | hog_channel | orientation  | pix_per_cell | spatial_feat | spatial_size | accuracy | feature vec. length | seconds to compute features | seconds to train SVC |
| :-------------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- |
| HLS | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.990 | 7968 | 12.58 | 10.48| 
| HSV | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.990 | 7968 | 12.14 | 9.18| 
| LUV | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.985 | 7968 | 11.77 | 0.69| 
| RGB | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.995 | 7968 | 12.50 | 8.08| 
| RGB | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.970 | 7968 | 11.66 | 8.32| 
| YCrCb | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.995 | 7968 | 67.31 | 8.90| 
| YUV | 2 | 64 | True | ALL | 8 | 8 | True | (32, 32) | 0.990 | 7968 | 12.12 | 0.78|   

> b. _YCrCb with various orientation_:   

| orientation | cell_per_block | color_space | hist_bins | hist_feat | hog_channel | pix_per_cell | spatial_feat | spatial_size | accuracy | feature vec. length | seconds to compute features | seconds to train SVC |
| :-------------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- |
| 6 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 1.000 | 7968 | 11.76 | 8.16| 
| 8 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 67.31 | 8.90| 
| 9 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 11.75 | 1.28| 
| 10 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.975 | 7968 | 11.82 | 8.14| 
| 12 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 11.87 | 8.71| 
| 14 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 12.11 | 9.89| 
| 16 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 11.72 | 8.63|    

> c. _RGB with various orientation_:   

| orientation | cell_per_block | color_space | hist_bins | hist_feat | hog_channel | pix_per_cell | spatial_feat | spatial_size | accuracy | feature vec. length | seconds to compute features | seconds to train SVC |
| :-------------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- |
| 6 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 0.985 | 7968 | 11.96 | 0.91| 
| 8 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 12.50 | 8.08| 
| 9 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 0.980 | 7968 | 12.01 | 8.33| 
| 10 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 0.980 | 7968 | 11.82 | 9.28| 
| 12 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 0.990 | 7968 | 11.90 | 8.99| 
| 14 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 0.985 | 7968 | 11.82 | 9.97| 
| 16 | 2 | RGB | 64 | True | ALL | 8 | True | (32, 32) | 1.000 | 7968 | 11.85 | 8.64| 

After several experiments, I finally chose to go with the following values that give both a good accuracy and a satisfactory sum of the feature computation time and the SVC training time for the tracker:   

| Parameters      |  Values  |
| :-------------- | :------- |
| cell_per_block  | 2       |
| color_space     | YCrCb   |
| hist_bins     | 64   |
| hist_feat     | True   |
| hog_channel     | ALL   |
| orientation     | 9       |
| pix_per_cell    | 8       |
| spatial_feat     | True   |
| spatial_size     | (32, 32) |

#### 3. Describe how you trained a classifier using your selected HOG features and color features

Before training the classifier, the combinaison of feature vectors need to be normalized. To do this, I used `sklearn.preprocessing.StandardScaler()` class to standardize features by removing the mean and scaling to unit variance. It is done to prevent machine learning estimators to behave badly.

```python
# -- lines 335 through 338 of the file main.py in the function classifier() --
X_scaler = StandardScaler().fit(X) # fit a per_column scaler
scaled_X = X_scaler.transform(X)   # apply the scaler to X (doc: "Perform standardization by centering and scaling")
```

Then I shuffled and split randomly the dataset of **17,761** `RGB` images in total (8,793 vehicle and 8,968 non-vehicle images) in two sets: 80% as training data and 20% as testing images. I used the `sklearn.model_selection.train_test_split()` function for this.

```python
# -- lines 343 through 345 of the file main.py in the function classifier() --
# split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)
# NB: the shuffle parameter is boolean and the default is always TRUE (see doc)
```

Finally, I trained a linear SVM using `sklearn.svm.LinearSVC()` function:

```python
# -- lines 351 through 355 of the file main.py in the function classifier() --
svc = LinearSVC() # use a linear SVC
svc.fit(X_train, y_train) 
```

And the trained classifier is saved on the hard drive in `data\pickled_object` as `svc.pkl file`:
```python
# -- lines 359 through 362 of the file main.py in the function classifier() --
# serialize/store svc:
joblib.dump(svc, args.pickled + 'svc.pkl')
```

I measured the mean accuracy on the test data and labels using `sklearn.svm.LinearSVC.score()` method:
```python
# -- lines 351 through 355 of the file main.py in the function classifier() --
# check the score of the SVC
accuracy = round(svc.score(X_test, y_test), 4)
print('Test Accuracy of SVC = ', accuracy)
```

The mean accuracy is about 99.5%:   

| orientation | cell_per_block | color_space | hist_bins | hist_feat | hog_channel | pix_per_cell | spatial_feat | spatial_size | accuracy | feature vec. length | seconds to compute features | seconds to train SVC |
| :-------------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- | :------- |
| 9 | 2 | YCrCb | 64 | True | ALL | 8 | True | (32, 32) | 0.995 | 7968 | 11.75 | 1.28| 

---
### Sliding Window Search
---
#### 1. Describe how you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?

The algorithm is looking for cars only in the lower half of the frame ([for now](https://goo.gl/yVx4gZ)). The scan is done with a overlapping of 0.5. I implemented a minimum and maximum scale `(0.75, 1.75)` and two additional and intermdiate scales to scan as well: `(1., 1.5)` in the function `multiscale_bboxes()`:
```python
# -- lines 473 through 478 of the file main.py --
def multiscale_bboxes(args, var, image):
    bboxes = []
    for var['scale'] in var['scales']:
        bboxes_scaled = find_cars(args, var, image)
        bboxes        = bboxes + bboxes_scaled
    return bboxes
```

Here is the visualization of the scanning of the window:
![fig.2](https://goo.gl/485qR3)

The algorithm is far from perfect. Nevertheless, the cars are dectected properly.

The whole process has been implemented in the function `find_cars()`. Unfortunately, it is slow. It could have been optimized by redesigning the scanning method as it was suggested during the lesson.

```python
# -- lines 383 through 470 of the file main.py --
def find_cars(args, var, image):
    # load the trained classifier
    X_scaler,_,svc = classifier(args, var, to_print=False) 
    bboxes = []
    img    = image.copy().astype(np.float32) / 255
    img_tosearch = img[var['y_start_stop'][0]:var['y_start_stop'][1], :, :]
    ctrans_tosearch = convert_color(img_tosearch, conv=var['color_space'])
    if var['scale'] != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch,
                          (np.int(imshape[1] / var['scale']), np.int(imshape[0] / var['scale'])))
    if var['hog_channel'] == 'ALL':
        ch1 = ctrans_tosearch[:, :, 0]
        ch2 = ctrans_tosearch[:, :, 1]
        ch3 = ctrans_tosearch[:, :, 2]
    else:
        ch1 = ctrans_tosearch[:, :, var['hog_channel']]
    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // var['pix_per_cell']) - var['cell_per_block'] + 1
    nyblocks = (ch1.shape[0] // var['pix_per_cell']) - var['cell_per_block'] + 1
    nfeat_per_block = var['orient'] * var['cell_per_block'] ** 2
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // var['pix_per_cell']) - var['cell_per_block'] + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    # Compute individual channel HOG features for the entire image
    if var['hog_channel'] == 'ALL':
        hog1 = get_hog_features(ch1, var['orient'], var['pix_per_cell'],
                                var['cell_per_block'], feature_vec=False)
        hog2 = get_hog_features(ch2, var['orient'], var['pix_per_cell'],
                                var['cell_per_block'], feature_vec=False)
        hog3 = get_hog_features(ch3, var['orient'], var['pix_per_cell'],
                                var['cell_per_block'], feature_vec=False)
    else:
        hog1 = get_hog_features(ch1, var['orient'], var['pix_per_cell'],
                                var['cell_per_block'], feature_vec=False)

    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos, xpos = yb * cells_per_step, xb * cells_per_step
            # Extract HOG for this patch
            if var['hog_channel'] == 'ALL': 
                hog_feat1 = hog1[ypos:ypos + nblocks_per_window,
                                 xpos:xpos + nblocks_per_window].ravel()
                hog_feat2 = hog2[ypos:ypos + nblocks_per_window,
                                 xpos:xpos + nblocks_per_window].ravel()
                hog_feat3 = hog3[ypos:ypos + nblocks_per_window,
                                 xpos:xpos + nblocks_per_window].ravel()
                hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            else:
                hog_features = hog1[ypos:ypos + nblocks_per_window,
                                    xpos:xpos + nblocks_per_window].ravel()
            ytop, xleft = ypos * var['pix_per_cell'], xpos * var['pix_per_cell']
            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop + window, xleft:xleft + window], (64, 64))
            # Get color features
            spatial_features = bin_spatial(subimg, size=var['spatial_size'])
            hist_features = color_hist(subimg, nbins=var['hist_bins'])
            # var['scale'] features and make a prediction
            test_features = X_scaler.transform(
                np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            # test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))
            test_prediction = svc.predict(test_features)
            if test_prediction == 1:
                xbox_left = np.int(xleft * var['scale'])
                ytop_draw = np.int(ytop * var['scale'])
                win_draw = np.int(window * var['scale'])
                bboxes.append(((xbox_left, ytop_draw+var['y_start_stop'][0]),
                               xbox_left+win_draw,ytop_draw+win_draw+var['y_start_stop'][0])))
    return bboxes 
```

#### 2. Show some examples of test images to demonstrate how your pipeline is working.  What did you do to optimize the performance of your classifier?

Ultimately I searched on **four** scales using `YCrCb` 3-channel HOG features plus spatially binned color and histograms of color in the feature vector, which provided a nice result.  Here are some example images:

![fig](https://goo.gl/jHmutU)

---
### Video Implementation
---
#### 1. The final video output
Here's the link to my video result: www.youtube.com/watch?v=vTU3D7OdSFY   

[![video.1](https://img.youtube.com/vi/vTU3D7OdSFY/0.jpg)](https://youtu.be/vTU3D7OdSFY)



#### 2. Describe how you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.
*The code for this part is contained in lines 482 through 536 of the file called `main.py`. It is split into several functions, i.e. `add_heat()`, `apply_threshold()`, `draw_labeled_bboxes()`, in lines 482 through 511. There are all combined in the function called `process_image()`.*   

I recorded the positions of positive detections in each frame of the video.
```python
# -- line 524 of the file main.py in the function process_image() --
    bboxes = multiscale_bboxes(args, var, image)
```
From the positive detections I created a heatmap to filter out false positives by increasing artificially the pixel values of all bounding boxes.
```python
# -- lines 351 through 355 of the file main.py in the function add_heat() --
def add_heat(heatmap, bbox_list):   
    for box in bbox_list:  # Iterate through list of bboxes
        # Add += 1 for all pixels inside each bbox = ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap
```
The more bounding boxes are overlapping, the higher the values are in those regions of the frame, meaning the classifier has recognized several times vehicle(s) in this region during the scan. And then thresholded that map to identify vehicle positions.
```python
# -- lines 493 through 496 of the file main.py in the function apply_threshold() --
def apply_threshold(heatmap, threshold):
    heatmap[heatmap <= threshold] = 0 # zeros out pixels below the threshold
    return heatmap
```

I recorded the positions of positive detections in each frame of the video. From the positive detections I created a heatmap and then thresholded that map to identify vehicle positions. I then used `scipy.ndimage.measurements.label()` to identify individual blobs in the heatmap. I then assumed each blob corresponded to a vehicle. I constructed bounding boxes to cover the area of each blob detected.

Here's an example result showing the heatmap from a series of frames of video, the result of `scipy.ndimage.measurements.label()` and the bounding boxes then overlaid on the last frame of video:   
![fig.0333](https://goo.gl/SDtMhC)

Here is the output of `scipy.ndimage.measurements.label()` on the integrated heatmap:
![fig.6](https://scontent-ams3-1.xx.fbcdn.net/v/t1.0-9/37695282_10155936612373772_5643635104807911424_n.jpg?_nc_cat=0&oh=25ff7502ff734944edcc0beeb8d0ec3d&oe=5C1426E7) 

The heapmap technique is very useful to reduce false positives. To Strengthen the algorithm, I reused an idea learnt in a previous project, i.e. I stored any new heat values and used an average of the last N entries. This last implementation cleaned up the remaining false positives.   
 
Here the resulting bounding boxes are drawn onto the last frame in the series:
![fig.0408](https://goo.gl/2oJGzX)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

I enjoyed learning about these techniques. I wish I had more free time to dig more about the sliding window search.   

We use a classifier as a base of the solution. Then I expect that the pipeline is as good as the quality and the diversity of the dataset. Certainly it will be difficult to generalize the use of this pipeline to any new environment. 

Futhermore, it is slow and resource demanding. By optimizing the sliding window search algorithm, I am sure that it would have a significant impact on the pipeline performance.   

I would love to get the same kind of video but with billboards on the sides during a motor show in town :)

---
ANNEXE
---

Here is the list of functions created and used to complete this project:

| Function  |  Line   |  Description                                            |
| :------------------------------| :-----: | :------------------------------------------------------ |
| `module\main.py\list_files()`  | 30-47 | list all files in the folder and save the list in output_images folder |
| `module\main.py\list_all_images()`  | 50-56 | list all `vehicle` and `non-vehicle` images and save the lists in `output_images` folder  |
| `module\main.py\get_hog_features()`  | 60-76 | return HOG features and visualization |
| `module\main.py\bin_spatial()`  | 80-85 | compute binned color features |
| `module\main.py\color_hist()`  | 89-97 | compute color histogram features |
| `module\main.py\extract_features()`  | 101-150 | extract features from a list of images |
| `module\main.py\slide_window()`  | 154-188 | take an image, start and stop positions in both x and y, window size (x and y dimensions), and overlap fraction (for both x and y) |
| `module\main.py\draw_boxes()`  | 192-200 | draw bounding boxes |
| `module\main.py\single_img_features()`  | 203-239 | extract features from a single image window (for a single image) |
| `module\main.py\search_windows()`  | 243-264 | pass an image and the list of windows to be searched |
| `module\main.py\visualize()`  | 280-295 | plot multiple images |
| `module\main.py\classifier()`  | 298-365 | return a trained Linear Support Vector Classification (svc) classifier and save it in `data\pickled_object` folder |
| `module\main.py\convert_color()`  | 368-380 | convert one image from one RGB color-space to another color-space (can be RGB HSV LUV HLS YUV YCrCb) |
| `module\main.py\find_cars()`  | 384-472 | extract features using hog sub-sampling and make predictions |
| `module\main.py\multiscale_bboxes()`  | 475-480 | extract features using hog sub-sampling and make predictions for various scales (0.75,1.,1.5, 1.75) |
| `module\main.py\add_heat()`  | 484-492 | add "heat" to a map for a list of bounding boxes |
| `module\main.py\apply_threshold()`  | 495-498 | threshold the map to reject areas affected |
| `module\main.py\draw_labeled_bboxes()`  | 502-513 | return the image labeled with bboxes |
| `module\main.py\process_image()`  | 529-553 | track vehicle in an image |
| `module\main.py\video()`  | 558-562 | replace a frame of a video by another image |
| `module\main.py\tracker_cars()`  | 565-570 | track vehicle in a video |
| `module\hog_parameters.py\exp_hog_parameters()`  | 19-135 | compute KPI for a various set of HOG parameters |